## Necessary packages are pandas and numpy

In [1]:
#Cell 1

import pandas as pd
import numpy as np

## Roadway EEC

In [3]:
#Cell 2
#Importing the CSV where the projects have been spatially joined with the roadway EEC as a dataframe


Roadway_EEC = pd.read_csv ("Roadway_EEC.csv", encoding ="latin-1")
Roadway_EEC.head ()

,ID,RT_UNIQUE,BEGIN_MP,END_MP,Roadway,Project_No,CHAF_ID,Route_uniq,BEGINNINGMILE,ENDINGMILE,...,EEC_2_CO,EB_NoF,EEC_NoF,EEC_2_NoF,EB_KAB_NoF,EEC_KAB_NoF,EEC_2_KAB_NoF,EB_CO_NoF,EEC_CO_NoF,EEC_2_CO_NoF
0,R2L-7676,004-US-0060 -000,13.630977,13.709,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,-0.339003,0.101663,-1.035071,-0.656160,0.048066,-0.083854,-0.039881,0.097506,-0.891132,-0.561586
1,R2L-7677,004-US-0060 -000,13.709000,13.731,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,0.703577,0.939240,0.618623,0.725495,0.649200,0.611991,0.624394,0.928875,0.650029,0.742977
2,R2L-7678,004-US-0060 -000,13.731000,13.807,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,-0.290141,0.099056,-1.008531,-0.639335,0.046834,-0.081704,-0.038858,0.095006,-0.868283,-0.547186
3,R2L-7679,004-US-0060 -000,13.807000,13.883,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,1.268076,1.920188,0.812600,1.181796,1.318119,1.189581,1.232427,1.897753,0.934465,1.255561
4,R2L-7680,004-US-0060 -000,13.883000,13.921,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,1.421851,1.870660,1.316866,1.501464,1.294702,1.230433,1.251856,1.850250,1.368606,1.529154


In [4]:
#Cell 3
#Adding a new column, Length. Segment Leghth = END_MP - BEGIN_MP

Roadway_EEC ["LENGTH"]=Roadway_EEC ["END_MP"]-Roadway_EEC["BEGIN_MP"]
Roadway_EEC.head (2)

,ID,RT_UNIQUE,BEGIN_MP,END_MP,Roadway,Project_No,CHAF_ID,Route_uniq,BEGINNINGMILE,ENDINGMILE,...,EB_NoF,EEC_NoF,EEC_2_NoF,EB_KAB_NoF,EEC_KAB_NoF,EEC_2_KAB_NoF,EB_CO_NoF,EEC_CO_NoF,EEC_2_CO_NoF,LENGTH
0,R2L-7676,004-US-0060 -000,13.630977,13.709,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,0.101663,-1.035071,-0.656160,0.048066,-0.083854,-0.039881,0.097506,-0.891132,-0.561586,0.078024
1,R2L-7677,004-US-0060 -000,13.709000,13.731,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,0.939240,0.618623,0.725495,0.649200,0.611991,0.624394,0.928875,0.650029,0.742977,0.022000


In [7]:
#Cell 4

#If the beginning milepoint is in the middle of a segment, it will get "S"
#If the ending milepoint is in the middle of a segment, it will get "E"
#If the beginning and ending mileipoint both fall in between a segment, it will get "M"
#Otherwise it will get "0"


def start_end(x):
    if x.BEGINNINGMILE>x.BEGIN_MP and x.ENDINGMILE <x.END_MP:
        return "M"
    elif x.BEGINNINGMILE>=x.BEGIN_MP and x.BEGINNINGMILE<=x.END_MP:
        return "S"
    elif x.ENDINGMILE>=x.BEGIN_MP and x.ENDINGMILE<=x.END_MP:
        return "E"
    else:
        return "O"

In [9]:
#Cell 5
#Creating new column for "S", "E", "M" and "O"

Roadway_EEC['StartEnd']=Roadway_EEC.apply (start_end, axis = 1)
Roadway_EEC.head ()

,ID,RT_UNIQUE,BEGIN_MP,END_MP,Roadway,Project_No,CHAF_ID,Route_uniq,BEGINNINGMILE,ENDINGMILE,...,EEC_NoF,EEC_2_NoF,EB_KAB_NoF,EEC_KAB_NoF,EEC_2_KAB_NoF,EB_CO_NoF,EEC_CO_NoF,EEC_2_CO_NoF,LENGTH,StartEnd
0,R2L-7676,004-US-0060 -000,13.630977,13.709,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,-1.035071,-0.656160,0.048066,-0.083854,-0.039881,0.097506,-0.891132,-0.561586,0.078024,O
1,R2L-7677,004-US-0060 -000,13.709000,13.731,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,0.618623,0.725495,0.649200,0.611991,0.624394,0.928875,0.650029,0.742977,0.022000,O
2,R2L-7678,004-US-0060 -000,13.731000,13.807,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,-1.008531,-0.639335,0.046834,-0.081704,-0.038858,0.095006,-0.868283,-0.547186,0.076000,O
3,R2L-7679,004-US-0060 -000,13.807000,13.883,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,0.812600,1.181796,1.318119,1.189581,1.232427,1.897753,0.934465,1.255561,0.076000,O
4,R2L-7680,004-US-0060 -000,13.883000,13.921,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,1.316866,1.501464,1.294702,1.230433,1.251856,1.850250,1.368606,1.529154,0.038000,O


In [10]:
#Cell 6

#Calculating the new length for which the EEC is needed.
#"S">> new_length = END_MP of segment - Beginning milepoint of project
#"E">> new_length = Ending milepoint of project - BEGIN_MP of the segment
# "M" >> new_length = Ending milepoint of project - Beginning milepoint of project
# "O">> new_length = 0


def new_length (x):
    if x.StartEnd=="S":
        A = x.END_MP - x.BEGINNINGMILE
        return A
    elif x.StartEnd=="E":
        B = x.ENDINGMILE - x.BEGIN_MP
        return B
    elif x.StartEnd=="M":
        C = x.ENDINGMILE - x.BEGINNINGMILE
        return C
    else:
        return 0

In [11]:
#Cell 7
#Adding column for new_length


Roadway_EEC['NewLength']=Roadway_EEC.apply (new_length, axis = 1)
Roadway_EEC.head ()

,ID,RT_UNIQUE,BEGIN_MP,END_MP,Roadway,Project_No,CHAF_ID,Route_uniq,BEGINNINGMILE,ENDINGMILE,...,EEC_2_NoF,EB_KAB_NoF,EEC_KAB_NoF,EEC_2_KAB_NoF,EB_CO_NoF,EEC_CO_NoF,EEC_2_CO_NoF,LENGTH,StartEnd,NewLength
0,R2L-7676,004-US-0060 -000,13.630977,13.709,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,-0.656160,0.048066,-0.083854,-0.039881,0.097506,-0.891132,-0.561586,0.078024,O,0.0
1,R2L-7677,004-US-0060 -000,13.709000,13.731,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,0.725495,0.649200,0.611991,0.624394,0.928875,0.650029,0.742977,0.022000,O,0.0
2,R2L-7678,004-US-0060 -000,13.731000,13.807,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,-0.639335,0.046834,-0.081704,-0.038858,0.095006,-0.868283,-0.547186,0.076000,O,0.0
3,R2L-7679,004-US-0060 -000,13.807000,13.883,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,1.181796,1.318119,1.189581,1.232427,1.897753,0.934465,1.255561,0.076000,O,0.0
4,R2L-7680,004-US-0060 -000,13.883000,13.921,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,1.501464,1.294702,1.230433,1.251856,1.850250,1.368606,1.529154,0.038000,O,0.0


In [13]:
# Cell 8

#Adding a new column for weighted EEC
# Calculating EEC for the new_length. Weighted EEC = (new_length/length of the segment)*EEC of the segment


Roadway_EEC ["Weighted_EEC"] = (Roadway_EEC["NewLength"]/Roadway_EEC["LENGTH"]) * Roadway_EEC ["Road_EEC"]
Roadway_EEC.head ()

,ID,RT_UNIQUE,BEGIN_MP,END_MP,Roadway,Project_No,CHAF_ID,Route_uniq,BEGINNINGMILE,ENDINGMILE,...,EB_KAB_NoF,EEC_KAB_NoF,EEC_2_KAB_NoF,EB_CO_NoF,EEC_CO_NoF,EEC_2_CO_NoF,LENGTH,StartEnd,NewLength,Weighted_EEC
0,R2L-7676,004-US-0060 -000,13.630977,13.709,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,0.048066,-0.083854,-0.039881,0.097506,-0.891132,-0.561586,0.078024,O,0.0,-0.0
1,R2L-7677,004-US-0060 -000,13.709000,13.731,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,0.649200,0.611991,0.624394,0.928875,0.650029,0.742977,0.022000,O,0.0,0.0
2,R2L-7678,004-US-0060 -000,13.731000,13.807,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,0.046834,-0.081704,-0.038858,0.095006,-0.868283,-0.547186,0.076000,O,0.0,-0.0
3,R2L-7679,004-US-0060 -000,13.807000,13.883,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,1.318119,1.189581,1.232427,1.897753,0.934465,1.255561,0.076000,O,0.0,0.0
4,R2L-7680,004-US-0060 -000,13.883000,13.921,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,1.294702,1.230433,1.251856,1.850250,1.368606,1.529154,0.038000,O,0.0,0.0


In [14]:
# Cell 9
# A new list of EEC with existing EEC and Weighted EEC

def new_EEC (x):
    if x.Weighted_EEC ==0:
        C = x.Road_EEC
        return C
    else:
        D = x.Weighted_EEC
        return D

In [16]:
# Cell 10
#New Column for Existing and Weighted EEC

Roadway_EEC ["NEW_EEC"] = Roadway_EEC.apply (new_EEC, axis = 1)
Roadway_EEC.head ()

,ID,RT_UNIQUE,BEGIN_MP,END_MP,Roadway,Project_No,CHAF_ID,Route_uniq,BEGINNINGMILE,ENDINGMILE,...,EEC_KAB_NoF,EEC_2_KAB_NoF,EB_CO_NoF,EEC_CO_NoF,EEC_2_CO_NoF,LENGTH,StartEnd,NewLength,Weighted_EEC,NEW_EEC
0,R2L-7676,004-US-0060 -000,13.630977,13.709,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,-0.083854,-0.039881,0.097506,-0.891132,-0.561586,0.078024,O,0.0,-0.0,-0.655581
1,R2L-7677,004-US-0060 -000,13.709000,13.731,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,0.611991,0.624394,0.928875,0.650029,0.742977,0.022000,O,0.0,0.0,0.624815
2,R2L-7678,004-US-0060 -000,13.731000,13.807,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,-0.081704,-0.038858,0.095006,-0.868283,-0.547186,0.076000,O,0.0,-0.0,-0.569858
3,R2L-7679,004-US-0060 -000,13.807000,13.883,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,1.189581,1.232427,1.897753,0.934465,1.255561,0.076000,O,0.0,0.0,0.980674
4,R2L-7680,004-US-0060 -000,13.883000,13.921,R2L,1,IP20000001,004-US-0060 -000,13.62,16.937,...,1.230433,1.251856,1.850250,1.368606,1.529154,0.038000,O,0.0,0.0,1.304277


In [17]:
#Cell 11
#Taking the summation of segment EECs for each project
#Project_No is unique for every project

Roadway_EEC_new = Roadway_EEC['NEW_EEC'].groupby(Roadway_EEC['Project_No']).sum ().to_frame(name='Roadway_EEC')
Roadway_EEC_new.head ()

,Roadway_EEC
Project_No,
1,22.443968
2,0.636327
3,-11.115524
4,6.525912
5,8.517202


In [18]:
#Cell 12
Roadway_EEC_new['Roadway_EEC'] = Roadway_EEC_new['Roadway_EEC'].replace(np.nan, 0)
Roadway_EEC_new.head ()

,Roadway_EEC
Project_No,
1,22.443968
2,0.636327
3,-11.115524
4,6.525912
5,8.517202


## Intersection EEC

In [21]:
#Cell 13
#Importing the spatially joined Intersection database

Intersection_EEC = pd.read_csv ("Intersection_EEC.csv", encoding ="latin-1")
Intersection_EEC.head ()

,StampIDs,count,MainRoute,MasterNode,MNodeMP,SecondRt,Project_No,CHAF_ID,Route_uniq,BEGINNINGMILE,...,Int_SPF,Int_EEC_2,Int_EB_KAB,Int_EEC_KAB,Int_SPF_KAB,Int_EEC_2_KAB,Int_EB_CO,Int_EEC_CO,Int_SPF_CO,Int_EEC_2_CO
0,49740,2,004-US-0060 -000,195851,13.612,004-CR-1001 -000,1,IP20000001,004-US-0060 -000,13.62,...,1.142272,1.392075,0.119236,-0.021871,0.141107,0.025165,2.035653,1.042501,0.993152,1.373552
1,49189,1,004-US-0060 -000,196607,16.780,004-KY-2532 -000,1,IP20000001,004-US-0060 -000,13.62,...,1.276544,1.524599,0.164344,0.000054,0.164290,0.054817,1.681628,0.409747,1.271881,0.833708
2,49316,1,004-US-0060 -000,317648,16.589,004-CR-1043 -000,1,IP20000001,004-US-0060 -000,13.62,...,1.137840,0.304375,0.117211,-0.021069,0.138280,0.025024,0.995852,0.004476,0.991376,0.334935
3,49364,1,004-US-0060 -000,317652,16.531,004-CR-1063 -000,1,IP20000001,004-US-0060 -000,13.62,...,2.038492,-0.027451,0.181385,-0.055963,0.237348,0.023153,1.267750,-0.521456,1.789207,0.074946
4,49577,1,004-US-0060 -000,196078,16.296,004-CS-4003 -000,1,IP20000001,004-US-0060 -000,13.62,...,1.150670,0.847461,0.118327,-0.021509,0.139836,0.025103,1.523072,0.520480,1.002592,0.854678


In [43]:
#Cell 14
#Taking the summation of Intersection EECs for each project
Intersection_EEC_new = Intersection_EEC['Int_EEC'].groupby(Intersection_EEC['Project_No']).sum ().to_frame(name='Intersection_EEC')
Intersection_EEC_new['Intersection_EEC'] = Intersection_EEC_new['Intersection_EEC'].replace(np.nan, 0)
Intersection_EEC_new.head ()

,Intersection_EEC
Project_No,
1,7.144030
2,0.387718
3,0.245900
4,-75.006657
5,-51.262813


In [32]:
# #Cell 15
# #Merging the Intersection EEC database with the Roadway EEC database

# Road_Int_EEC = pd.merge (Intersection_EEC_new,Roadway_EEC_new,left_on = "Project_No",right_on = "Project_No", how = "outer")
# Road_Int_EEC.head ()

## Ramp EEC

In [33]:
#Cell 15
#Importing the spatially joined Ramp database

Ramp_EEC = pd.read_csv ("Ramp_EEC.csv", encoding ="latin-1")
Ramp_EEC.head ()

,Project_No,CHAF_ID,Route_ID,BMP,EMP,EB,Ramp_EEC,SPF,EEC_2,EB_KAB,EEC_KAB,SPF_KAB,EEC_2_KAB,EB_CO,EEC_CO,SPF_CO,EEC_2_CO,w_KAB,w_CO
0,65,IP20150149,047-I -0065 -211,0.0,0.28,4.030103,-0.281651,4.311754,1.155600,0.091049,-0.146853,0.237902,-0.067553,3.999512,0.015697,3.983815,1.343635,0.382715,0.030155
1,66,IP20150149,047-I -0065 -221,0.0,0.34,1.356577,-4.639560,5.996137,-2.640848,0.114440,-0.205086,0.319527,-0.098577,1.119587,-4.441031,5.560618,-2.587492,0.358156,0.026222
2,67,IP20150149,047-I -0065 -222,0.0,0.02,0.210558,-0.177140,0.387698,-0.047908,0.007412,-0.013283,0.020695,-0.006385,0.009444,-0.350702,0.360146,-0.230653,0.358156,0.026222
3,68,IP20150149,047-I -0065 -231,0.0,0.29,18.008096,12.295578,5.712518,14.199751,0.102366,-0.195694,0.298060,-0.096340,18.670635,13.359669,5.310965,15.129991,0.343442,0.024061
4,69,IP20150149,047-I -0065 -241,0.0,0.36,2.292252,-3.973727,6.265979,-1.885068,0.120698,-0.214293,0.334991,-0.102629,2.101148,-3.708761,5.809909,-1.772125,0.360302,0.026549


In [45]:
#Cell 16
#Taking the summation of Ramp EECs for each project
Ramp_EEC_new = Ramp_EEC['Ramp_EEC'].groupby(Ramp_EEC['Project_No']).sum ().to_frame(name='Ramp_EEC')
Ramp_EEC_new['Ramp_EEC'] = Ramp_EEC_new['Ramp_EEC'].replace(np.nan, 0)
Ramp_EEC_new.head ()

,Ramp_EEC
Project_No,
65,-0.281651
66,-4.639560
67,-0.177140
68,12.295578
69,-3.973727


## Merging Projects, Roadway_EEC, Intersection_EEC and Ramp_EEC database

In [46]:
#Cell 17
#Importing the file with project details.

Projects = pd.read_csv ("Projects.csv", encoding ="latin-1")
Projects.head ()

,Project_No
0,1
1,2
2,3
3,4
4,5


In [47]:
#Cell 18
#Merge 1: Projects and Roadway_EEC


Merge1 = pd.merge (Projects,Roadway_EEC_new,left_on = "Project_No",right_on = "Project_No", how = "outer")
Merge1.head ()

,Project_No,Roadway_EEC
0,1,22.443968
1,2,0.636327
2,3,-11.115524
3,4,6.525912
4,5,8.517202


In [48]:
#Cell 19
#Merge 2: Merge1 and Intersection EEC


Merge2 = pd.merge (Merge1,Intersection_EEC_new,left_on = "Project_No",right_on = "Project_No", how = "outer")
Merge2.head ()

,Project_No,Roadway_EEC,Intersection_EEC
0,1,22.443968,7.144030
1,2,0.636327,0.387718
2,3,-11.115524,0.245900
3,4,6.525912,-75.006657
4,5,8.517202,-51.262813


In [56]:
#Cell 20
#Merge 2: Merge1 and Intersection EEC


Merge3 = pd.merge (Merge2,Ramp_EEC_new,left_on = "Project_No",right_on = "Project_No", how = "outer")
Merge3 = Merge3.replace(np.nan, 0)
Merge3.head ()

,Project_No,Roadway_EEC,Intersection_EEC,Ramp_EEC
0,1,22.443968,7.144030,0.0
1,2,0.636327,0.387718,0.0
2,3,-11.115524,0.245900,0.0
3,4,6.525912,-75.006657,0.0
4,5,8.517202,-51.262813,0.0


In [57]:
#Cell 21
#Total EEC of a project =  Roadway EEC + Intersection EEC + Ramp_EEC


Merge3 ["Final_EEC"] = Merge3 ["Intersection_EEC"] + Merge3 ["Roadway_EEC"] + Merge3 ["Ramp_EEC"]
Merge3.head ()

,Project_No,Roadway_EEC,Intersection_EEC,Ramp_EEC,Final_EEC
0,1,22.443968,7.144030,0.0,29.587998
1,2,0.636327,0.387718,0.0,1.024045
2,3,-11.115524,0.245900,0.0,-10.869624
3,4,6.525912,-75.006657,0.0,-68.480745
4,5,8.517202,-51.262813,0.0,-42.745611


## Sorting and Ranking

In [60]:
#Cell 22
#Sorting the EECs from highest to lowest

FINAL_EEC_sorted = Merge3.sort_values(by= "Final_EEC", ascending=False)
FINAL_EEC_sorted.head ()

,Project_No,Roadway_EEC,Intersection_EEC,Ramp_EEC,Final_EEC
237,238,932.393616,72.381994,0.0,1004.775611
911,912,377.887710,434.188819,0.0,812.076529
119,120,236.249255,419.126769,0.0,655.376024
277,278,321.019999,236.012679,0.0,557.032678
908,909,246.750835,227.479723,0.0,474.230558


In [69]:
#Cell 23
#Adding Rank

FINAL_EEC_sorted["Rank"] = FINAL_EEC_sorted["Final_EEC"].rank(ascending=False)
FINAL_EEC_sorted.head ()

,Project_No,Roadway_EEC,Intersection_EEC,Ramp_EEC,Final_EEC,Rank
237,238,932.393616,72.381994,0.0,1004.775611,1.0
911,912,377.887710,434.188819,0.0,812.076529,2.0
119,120,236.249255,419.126769,0.0,655.376024,3.0
277,278,321.019999,236.012679,0.0,557.032678,4.0
908,909,246.750835,227.479723,0.0,474.230558,5.0


In [34]:
# Cell 24
FINAL_EEC_sorted.to_csv ("Final EEC List.csv")

## Additional Analysis
### Summary Statistics

In [35]:
#Cell 25
FINAL_EEC_sorted["Final_EEC"].describe ()

count    1511.000000
mean       19.868551
std        81.872324
min      -282.954773
25%        -7.323023
50%         0.783589
75%        19.579806
max      1122.996107
Name: Final_EEC, dtype: float64

### Sorting by sign of EEC

In [70]:
#Cell 26
def SIGN (x):
    if x.Final_EEC >0:
        return "P"
    elif x.Final_EEC <0:
        return "N"
    else:
        return "0"

In [72]:
#Cell 27
FINAL_EEC_sorted ["Sign"] = FINAL_EEC_sorted.apply (SIGN, axis = 1)
FINAL_EEC_sorted.head ()

,Project_No,Roadway_EEC,Intersection_EEC,Ramp_EEC,Final_EEC,Rank,Sign
237,238,932.393616,72.381994,0.0,1004.775611,1.0,P
911,912,377.887710,434.188819,0.0,812.076529,2.0,P
119,120,236.249255,419.126769,0.0,655.376024,3.0,P
277,278,321.019999,236.012679,0.0,557.032678,4.0,P
908,909,246.750835,227.479723,0.0,474.230558,5.0,P


In [74]:
#Cell 28
FINAL_EEC_sorted["Sign"].value_counts ()


P    910
N    781
0     39
Name: Sign, dtype: int64